In [ ]:
# for the LLM

!pip3 install openai

In [ ]:
# only for OCR 

#!pip3 install langdetect
#!pip3 install pytesseract
#!pip3 install pdf2image Pillow

In [4]:
import pandas as pd
import re
from openai import OpenAI

## 1. Initial Text Extraction

#### Using PyMuPDF

In [45]:
import fitz  # Import the PyMuPDF library

def extract_text_pymupdf(pdf_path):
    doc = fitz.open(pdf_path)  # Open the PDF file
    text = ""  # Initialize an empty string to store text

    for page in doc:  # Iterate through each page
        text += page.get_text()  # Extract text from the page and append it

    doc.close()  # Close the document
    return text

# here change path to pdf
#pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-DG-Press-HoldinG-B.V.pdf'
pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-Momo-Medical-Holding-B.V.-2022.pdf'
#pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-Informed-IT-Holding-B.V.pdf'
#pdf_path = 'KVK Sample Files 2 - Julia/01016572-docType-sd_jaarrek_art394_lid1-docJaar-2022-docCreatie-2023-12-21-docId-090299cc61e0f819.pdf'
raw_text = extract_text_pymupdf(pdf_path)
text = extract_text_pymupdf(pdf_path)
text_data = extract_text_pymupdf(pdf_path)
print(text)

The accountant's compilation report has been issued
 
 
 
 
 
Consolidated financial statements 2022
Momo Medical Holding B.V.
DELFT
Table of contents
Consolidated balance sheet . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2
Consolidated profit and loss account . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
Notes to the consolidated financial statements . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5
Balance sheet . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 14
Profit and loss account . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 16
Notes to the financial statements . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 17
Specifications to the financial statements . . . . . . . . . . . . . . . . . . . . . . . . 

### Testing OCR

In [364]:
# need to install poppler and tesseract (brew install poppler tesseract)
import pytesseract
from PIL import Image
from pdf2image import convert_from_path

def extract_text_from_pdf(pdf_path):
    # Convert PDF to a list of images
    pages = convert_from_path(pdf_path, 300)  # DPI set to 300 for better OCR accuracy
    all_text = ''
    
    # both English and Dutch 
    lang = 'eng+nld'
    
    # NOTE: mb just put into a list instead of concating with \n
    for page in pages:
        text = pytesseract.image_to_string(page, lang=lang)
        all_text += text + '\n'  # Append text of each page with a newline

    return all_text

# here change path to pdf
pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-DG-Press-HoldinG-B.V.pdf'
# pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-Momo-Medical-Holding-B.V.-2022.pdf'
# pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-Informed-IT-Holding-B.V.pdf'
# pdf_path = 'KVK Sample Files 2 - Julia/01042818-docType-sd_jaarrek_art394_lid1-docJaar-2022-docCreatie-2023-12-28-docId-090299cc626872c5.pdf'
# pdf_path = 'KVK Sample Files 2 - Julia/01016572-docType-sd_jaarrek_art394_lid1-docJaar-2022-docCreatie-2023-12-21-docId-090299cc61e0f819.pdf'
text = extract_text_from_pdf(pdf_path)
text_data= extract_text_from_pdf(pdf_path)
print(text)


DG press HoldinG B.V.
Statutair gevestigd te Brummen
Rapport inzake de jaarverslaggeving 2022

JAARREKENING

Gewaarmerkt voor ioertüúficatiedesleinden
Visser & Visser Audit en Assurance B.V

3 1 MEI 2023

1.1
1.2
1.3
1.4

3.1
3.2
3.3

3.4
3.5
3.6

4.1
4.2

4.3
4.4
45

5.1
5.2
5.3

INHOUDSOPGAVE

RAPPORT

Algemeen

Geconsolideerde resultaten
Financiéle positie

Fiscale positie

BESTUURSVERSLAG

Bestuursverslag

GECONSOLIDEERDE JAARREKENING
Geconsolideerde balans per 31 december 2022
Geconsolideerde winst-en-verliesrekening over 2022

Geconsolideerd kasstroomoverzicht over 2022

Grondslagen van waardering en resultaatbepaling van de geconsolideerde jaarreken
Toelichting op de geconsolideerde balans per 31 december 2022

Toelichting op de geconsolideerde winst-en-verliesrekening over 2022

ENKELVOUDIGE JAARREKENING
Enkelvoudige balans per 31 december 2022

Enkelvoudige winst-en-verliesrekening over 2022

Grondslagen van waardering en resultaatbepaling van de enkelvoudige jaarrekening
Toel

In [367]:
# this part is responsible for cleaning the OCR extracted text and adjusting the format such that it is ready for furter processing

def data_cleaning(text_data):
    def is_year(s):
        try:
            num = int(s.replace(',', '').replace('.', ''))
            return 2000 <= num <= 2025
        except ValueError:
            return False

    def delete_year_numbers(line):
        words = line.split()
        return ' '.join(word for word in words if not is_year(word))
    
    def remove_small_numbers(line):
        def should_remove(word):
            try:
                # Check if word is a number (integer or float) and less than 100
                num = float(word)
                return num < 100
            except ValueError:
                return False
        words = line.split()
        return ' '.join(word for word in words if not (len(word) <= 2 or should_remove(word)))


    def remove_percentages(line):
        return re.sub(r'\b\d{1,2}\b%', '', line)
    
    def remove_brackets(line):
        return re.sub(r'\((\d+)\)', r'-\1', line) 

    def is_page_number(line):
        return line.strip().startswith("Pagina")
    
    #remove all '_" signs from the text and numbers
    def remove_underscores(line):
        return line.replace('_', ' ')
    
    #replace "-" and ":" with 0  (sometimes - is read as : )
    def replace_dash_with_zero(line):
        return line.replace('-', '0').replace(':', '0')

    def remove_currency_symbols(line):
        return line.replace('€', '').replace('$', '')
    
    # Split the text into the first 1500 characters and the rest
    initial_segment = text_data[:1500]
    rest_of_text = text_data[1500:]
    initial_segment = remove_small_numbers(initial_segment)
    lines = (initial_segment + rest_of_text).split('\n')


    cleaned_lines = []
    lines = text_data.split('\n')

    for line in lines:
        if not is_page_number(line):
            line = remove_small_numbers(line)
            line = delete_year_numbers(line)
            line = remove_percentages(line)
            line = remove_currency_symbols(line)
            line = remove_underscores(line)
            line = replace_dash_with_zero(line)
            line = remove_brackets(line)
            cleaned_lines.append(line)

    cleaned_data = '\n'.join(cleaned_lines)
    return cleaned_data


cleaned_data = data_cleaning(text_data)
print(cleaned_data)

press HoldinG B.V.
Statutair gevestigd Brummen
Rapport inzake jaarverslaggeving

JAARREKENING

Gewaarmerkt voor ioertüúficatiedesleinden
Visser Visser Audit Assurance B.V

MEI

























INHOUDSOPGAVE

RAPPORT

Algemeen

Geconsolideerde resultaten
Financiéle positie

Fiscale positie

BESTUURSVERSLAG

Bestuursverslag

GECONSOLIDEERDE JAARREKENING
Geconsolideerde balans per december
Geconsolideerde winst0en0verliesrekening over

Geconsolideerd kasstroomoverzicht over

Grondslagen van waardering resultaatbepaling van geconsolideerde jaarreken
Toelichting geconsolideerde balans per december

Toelichting geconsolideerde winst0en0verliesrekening over

ENKELVOUDIGE JAARREKENING
Enkelvoudige balans per december

Enkelvoudige winst0en0verliesrekening over

Grondslagen van waardering resultaatbepaling van enkelvoudige jaarrekening
Toelichting enkelvoudige balans per december

Toelichting enkelvoudige winst0en0verliesrekening over

OVERIGE GEGEVENS
Statutaire regeling omtrent bestem

In [354]:

# Text just for testing
sample_text = """

Projectopbrengsten
Brutomarge

Kosten van grond en hulpstoffen
Kosten van uitbesteed werk

Lonen en Salarissen
Sociale lasten

Overige personeelskosten
Afschrijvingen

Overige bedrijfskosten
Totaal bedrijfskosten

Bedrijfsresuitaat

Andere rentebaten en soortgelijke opbrengsten

Rentelasten en soortgelijke kosten
Resultaat voor belastingen

Vennootschapsbelasting

Resultaat na belastingen

11.121.880
11.121.880

5.685.632
298.405
2.713.213
694.589
292.192
67.466
1.096.123
10.847.620

274.260

0

-187,548
86.712

2.963

89.675
"""
#TODO add condition for more than 1 numeric value in a line

# Split the text into lines and remove empty lines
lines = [line.strip() for line in sample_text.split('\n') if line.strip()]

# Separate descriptions and numbers
descriptions = []
numbers = []
for line in lines:
    if re.match(r'^[\d,.\-]+$', line):
        numbers.append(line.replace(',', ''))
    else:
        descriptions.append(line)

# Determine the minimum length to avoid index errors
min_length = min(len(descriptions), len(numbers))

# Slice descriptions list to take from the bottom if there are more descriptions
if len(descriptions) > len(numbers):
    descriptions = descriptions[-min_length:]

# Print descriptions and corresponding numbers in columns
for i in range(min_length):
    print(f"{descriptions[i]:40} {numbers[i]:>15}")

# Print a warning if there is a mismatch between descriptions and numbers
if len(descriptions) != len(numbers):
    print("\nWarning: Mismatch between the number of descriptions and numbers.")
    print(f"Descriptions count: {len(descriptions)}")
    print(f"Numbers count: {len(numbers)}")


Projectopbrengsten                            11.121.880
Brutomarge                                    11.121.880
Kosten van grond en hulpstoffen                5.685.632
Kosten van uitbesteed werk                       298.405
Lonen en Salarissen                            2.713.213
Sociale lasten                                   694.589
Overige personeelskosten                         292.192
Afschrijvingen                                    67.466
Overige bedrijfskosten                         1.096.123
Totaal bedrijfskosten                         10.847.620
Bedrijfsresuitaat                                274.260
Andere rentebaten en soortgelijke opbrengsten               0
Rentelasten en soortgelijke kosten               -187548
Resultaat voor belastingen                        86.712
Vennootschapsbelasting                             2.963
Resultaat na belastingen                          89.675


In [338]:
# another table type

def process_table_lines(text_data):
    lines = text_data.strip().split('\n')
    table_lines = []

    for line in lines:
        words = line.split()
        description = []
        values = []

        for word in words:
            if word.replace('.', '', 1).replace('-', '', 1).isdigit() or word == '_':
                values.append(word)
            else:
                description.append(word)
        
        # Skip line if there are no numeric values next to the text
        if len(values) == 0:
            continue
        
        # skip the line if there is no text before the numbers (it's totals from the subcategories)
        if len(description) == 0:
            continue
        
        description_text = ' '.join(description)
        table_lines.append((description_text, values))

    return table_lines

def format_table(table_lines):
    formatted_table = []

    # Header
    formatted_table.append("Description\tYear_1_Value\tYear_2_Value")

    # Body
    for description, values in table_lines:
        num_values = len(values)
        if num_values == 0:
            formatted_table.append(f"{description}\t\t")
        elif num_values == 1:
            formatted_table.append(f"{description}\t{values[0]}\t")
        elif num_values == 2:
            formatted_table.append(f"{description}\t{values[0]}\t{values[1]}")
        else:
            formatted_table.append(f"{description}\t{values[0]}\t{values[1]}\t{' '.join(values[2:])}")

    return formatted_table


cleaned_data = """
Hoofdsom 154.408 154.408
Cumulatieve aflossing -60.048 -46.704
Saldo per 01-01-2021 94.360 107.704
Aflossing 13344 _ 23344
Stand per 31-12-2021
Hoofdsom 154.408 154.408
Cumulatieve aflossing -73.392 -60.048
Kortlopend deel -13.344 -13.344
Stand per 31-12-2021 67,672 81.016
Kortlopende schulden 31-12-2021 31-12-2020
"""

table_lines = process_table_lines(cleaned_data)
formatted_table = format_table(table_lines)
for line in formatted_table:
    print(line)

Description	Year_1_Value	Year_2_Value
Hoofdsom	154.408	154.408
Cumulatieve aflossing	-60.048	-46.704
Saldo per 01-01-2021	94.360	107.704
Aflossing	13344	_	23344
Hoofdsom	154.408	154.408
Cumulatieve aflossing	-73.392	-60.048
Kortlopend deel	-13.344	-13.344
Stand per 31-12-2021 67,672	81.016	


In [355]:
# proposed setup??
# TABLE TYPES
#TABLE_TYPE_1 = 1 # perfectly parsed 
#TABLE_TYPE_2 = 2 # labels separated from numbers
# ...
# TODO: determine number of lines needed to detect (make it a global constant or pass to function)
#def detect_table_type(lines, num_lines=8):

In [288]:
# testing adjusting logic from previous approach, for now testing different approach but left in case it is needed
 

MAX_DESCRIPTION_LENGTH = 220

def remove_small_numbers(text, threshold=100, char_limit=1000):
    initial_segment = text[:char_limit]
    rest_of_text = text[char_limit:]
    modified_segment = re.sub(r'\b\d{1,2}\b', '', initial_segment)  # Remove numbers less than 100
    return modified_segment + rest_of_text

def is_year(s: str) -> bool:
    try:
        num = int(s.replace(',', '').replace('.', ''))
        return 2000 <= num <= 2025
    except ValueError:
        return False

def is_page_number(s: str) -> bool:
    return s.strip().startswith("Page") or s.strip().startswith("Pagina")

def extract_numbers_and_text(line):
    parts = re.split(r'(\b[-+]?\d{1,3}(?:,\d{3})*\.?\d*%?\b)', line)
    numbers = [part for part in parts if re.match(r'^[-+]?\d{1,3}(?:,\d{3})*\.?\d*%?$', part) and '%' not in part]
    text_parts = [part for part in parts if not re.match(r'^[-+]?\d{1,3}(?:,\d{3})*\.?\d*%?$', part)]
    return ' '.join(text_parts).strip(), numbers

def parse_adjusted_financial_text(text):
    text = remove_small_numbers(text) # remove small numbers from begining of text
    # first try to detect which table type we're currently dealing with:
    #   1. Perfectly parsed data (eg. Machinery 44,292 13,719, ...)
    #   2. Properly parsed table but labels separated from numbers
    #   3. ??
    
    lines = text.strip().split("\n").
    parsed_data = []
    misc_text_data = []
    current_description = ""
    no_numeric_data_on_page = True

    for line in lines:
        if is_page_number(line):
            if no_numeric_data_on_page and current_description:
                misc_text_data.append(current_description)
            current_description = ""
            no_numeric_data_on_page = True
            continue

        text, numbers = extract_numbers_and_text(line)
        filtered_numbers = [num for num in numbers if not is_year(num)]

        if numbers:
            if len(filtered_numbers) > 0:
                no_numeric_data_on_page = False
                if current_description and len(current_description) <= MAX_DESCRIPTION_LENGTH:
                    # Ensure each row has exactly three elements (one description and two numeric values)
                    row = [current_description.lower()] + filtered_numbers[:2]
                    if len(row) < 3:  # Pad with None if necessary
                        row += [None] * (3 - len(row))
                    parsed_data.append(row)
                current_description = ""
            current_description += " " + text if current_description else text
        elif len(current_description) + len(text) <= MAX_DESCRIPTION_LENGTH:
            current_description += " " + text

    if no_numeric_data_on_page and current_description:
        misc_text_data.append(current_description)


    return pd.DataFrame(parsed_data, columns=['Description', 'Year 1 EUR', 'Year 2 EUR']), misc_text_data

# Parse the adjusted text data
df_parsed, misc_text_data = parse_adjusted_financial_text(text_data)

# Display the parsed DataFrame
display(df_parsed)


,Description,Year 1 EUR,Year 2 EUR
0,"momo medical holding b.v., delft consolidated balance sheet as at december after appropriation of results eur eur assets non-current assets property, plant and equipment",292,719
1,"machinery , ,","784,641","214,361"
2,other tangible assets,"828,933","228,080"
3,"total of property, plant and equipment","828,933","228,080"
4,total of non-current assets current assets inventories,"266,885",896
5,"raw materials and consumables ,",450,064
6,"inventories under construction , ,","125,005",284
7,"finished and trade goods ,","434,340","150,244"
8,total of inventories receivables,"182,558","229,329"
9,trade receivables,50,50


## 2. Extracting, Categorising, and Processing data 

In [46]:
MAX_DESCRIPTION_LENGTH = 220
LLM_length=100

# for the first 1000 characters of the text, delete all numeric values that are smaller than 100
# this is to avoid extracting page numbers and other irrelevant data
# is_number returns a number stripped from commas, dots and parentheses and a boolean indicating if the input is a number
# or the raw data if `s` is not a number
def is_number(s: str) -> [str, bool]:
    if s.startswith("(") and s.endswith(")"):
        s = s[1:-1]
    elif s == "-":
        return 0, True
    # NOTE: more cases here
    num = s.replace(",", "").replace(".", "")
    return num, num.isdigit()

def is_year(s) -> bool: return 2000 <= int(s) <= 2025

def ends_with_percent(s: str) -> bool: return s.endswith("%")

def is_page_number(s: str) -> bool:   return s.strip().startswith("Page") or s.strip().startswith("Pagina") 

# function added to skip over the table of content page numbers
def remove_small_numbers(text, threshold=100, char_limit=1000):
    # Limit the text to the first 1000 characters for processing
    initial_segment = text[:char_limit]
    # Remove numbers smaller than 100
    modified_segment = re.sub(r'\b\d{1,2}\b', '', initial_segment)
    
    return modified_segment + text[char_limit:]


def parse_adjusted_financial_text(text):
    text = remove_small_numbers(text)
     
    # Split the text into lines
    lines = text.strip().split("\n")
    
    # Initialize an empty list to store our parsed data
    parsed_data = []
    
    # this variable tracks if we found any numeric data on the current page
    no_numeric_data_on_page = True
    misc_text_data = []
    
    # Initialize empty strings for current description and values
    current_description = ""
    value_year1 = ""
    
    # Iterate over each line in the text
    for line in lines:
        # Check if line is a description or a value
        maybe_num, is_num = is_number(line)
        
        if is_num and is_year(maybe_num):
            continue
        elif ends_with_percent(line):
            continue
        
        if is_num:
            # If it's a digit, it's a value
            if value_year1 == "":
                # If the first value is not yet set, set it as year1 value
                value_year1 = maybe_num
            else:
                print([current_description.strip().lower(), int(value_year1), int(maybe_num)])
                parsed_data.append([current_description.strip().lower(), int(value_year1), int(maybe_num)])
                # Reset the values for the next set of data
                current_description = ""
                value_year1 = ""
                no_numeric_data_on_page = False
        else:
            # we might find long streches of text unrelated to data labeling
            # in that case we have the following solutions:
            #  * cap descripiton length (x chars)
            #  * try to extract category name using llms
            # If it's not a digit, it's a description
            # Accumulate descriptions until we reach a digit
            if current_description:
                if is_page_number(line):
                    if no_numeric_data_on_page:
                        misc_text_data.append(current_description)
                        current_description = ""
                    # we want to reset the tracker at every new page
                    no_numeric_data_on_page = True
                else: current_description += " " + line.strip()
               
                #skip if the length is more than MAX_DESCRIPTION_LENGTH
                if len(current_description) > MAX_DESCRIPTION_LENGTH:
                    current_description = ""        
            else:
                current_description = line
                
    # Convert the parsed data into a DataFrame
    return pd.DataFrame(parsed_data, columns=['Description', 'Year 1 EUR', 'Year 2 EUR']), misc_text_data

In [ ]:
# Parse the adjusted text data
df_parsed, misc_text_data = parse_adjusted_financial_text(text_data)

# Display the parsed DataFrame
#display(df_parsed)


In [7]:
# TODO: different way to extract information -> from file names, this only works correctly for English documents, new approach in cell below
# extracting company information
client = OpenAI(base_url = 'http://localhost:11434/v1',api_key='ollama')

def prepare_prompt(text: str) -> list:
    return [
        # prompt not final, to be adjusted
        {"role": "system", "content": " extract the name of the company, the two years that the financial statement is about (i.e 2021, 2022), in the order they appear, the currency, and the type of financial statement. Output 5 (not more) variables, separated by commas."},
        {"role": "user", "content": text},
        {"role": "assistant", "content": "category name:"}
    ]


def output_company_information(text: str) -> str:
    messages = prepare_prompt(text)
    response = client.chat.completions.create(
        # TODO: try on smaller models
        model="gemma:7b",
        messages = messages,
        temperature=0.4
    )
  
    company_information = response.choices[0].message.content
    return company_information

# from the first 500 characters of df_parsed, extract company name, year, currency, and the type of financial statement
company_information = output_company_information(text[:500])
#make company informarion only the first row of the output
company_information = company_information.split("\n")[0]
print(company_information)


# make a new dataframe with the extracted company information
company_information = company_information.split(",")
company_information_df = pd.DataFrame([company_information], columns=['Company Name', 'Year 1', 'Year 2', 'Currency', 'Type of Financial Statement'])
display(company_information_df)

 Momo Medical Holding B.V., 2022, 2021, EUR, Consolidated financial statements


,Company Name,Year 1,Year 2,Currency,Type of Financial Statement
0,Momo Medical Holding B.V.,2022,2021,EUR,Consolidated financial statements


In [48]:
### getting company info from file name

#get the file name from path
file_name = pdf_path.split("/")[-1]
#transform the file name to lower case
file_name = file_name.lower()
#remove the words pdf, Jaarverslag and Jaarrekening,... from the file name
file_name = file_name.replace(".pdf", "").replace("jaarverslag", "").replace("jaarrekening", "").replace("geconsolideerd", "").replace("geconsolideerde", "").replace("annual", "").replace("report", "").replace("consolidated", "").replace("financial", "").replace("statement", "").replace("statements", "") .replace("jaarbericht", "") 

# if there is a number in the file name, it is the year
year = re.findall(r'\d{4}', file_name)
# company name is the file name without the year
company_name = file_name.replace(year[0], "").replace("-", " ").strip()

#from the first 1000 characters of the text, extract the currency
#TODO: add more currencies, improve logic
currency = re.findall(r'€|usd|dollar', text[:1000].lower())
if currency:
    currency = currency[0]
else:
    currency = "EUR"

#year is Year_1, Year_2 is the year -1, display the extracted company name and year 1 and 
company_information_df = pd.DataFrame([[company_name, year[0], str(int(year[0])-1), currency ]], columns=['Company Name', 'Year 1', 'Year 2', 'Currency'])


display(company_information_df)


,Company Name,Year 1,Year 2,Currency
0,momo medical holding b.v.,2022,2021,EUR


In [49]:
# setup for replacing all outputs longer than XXX characters with a category name extracted by gemma

def prepare_messages(text: str) -> list:
    return [
        # prompt not final, to be adjusted
        {"role": "system", "content": "This is a fragment of description from a financial statement. Extract a financial category name from the description. Output only the category and no other text or explanations. If there is no category or not applicable, output only: -."},
        {"role": "user", "content": text},
        {"role": "assistant", "content": "category name:"}
    ]

def output_category_name(text: str) -> str:
    messages = prepare_messages(text)
    response = client.chat.completions.create(
        # TODO: try on smaller models
        model="gemma:7b",
        messages = messages,
        temperature=0.8
    )
    category_name = response.choices[0].message.content
     # convert to lowercase
    category_name = category_name.lower()
    # remove whitespaces
    category_name = category_name.strip()
    return category_name


In [50]:
#change the 'Year 1 EUR' and 'Year 2 EUR' columns in df_parsed to the values under Year 1 and Year 2 in company_information_df
df_parsed = df_parsed.rename(columns={'Year 1 EUR': company_information_df['Year 1'][0], 'Year 2 EUR': company_information_df['Year 2'][0]})
df_parsed['Category'] = df_parsed['Description']

# extracring category names for descriptions longer than LLM_length
for i, row in enumerate(df_parsed['Description']):
    if len(row) > LLM_length and len(row) < MAX_DESCRIPTION_LENGTH:
        category_name = output_category_name(row)
        # tentative solution, to be deleted once the prompt is adjusted
        category_name = category_name.split("\n")[0]
        df_parsed.at[i, 'Category'] = category_name
        print(row, "->", category_name)
  


page 2 momo medical holding b.v., delft consolidated balance sheet as at 31 december 2022 after appropriation of results eur eur assets non-current assets property, plant and equipment   machinery -> assets
current receivables from other legal entities and companies with a participating interest in the legal entity or from participating interests of the legal entity -> current receivables from other legal entities and companies with a participating interest in the legal entity or from participating interests of the legal entity
page 4 momo medical holding b.v., delft consolidated profit and loss account 2022 eur eur total operating income   net revenue -> total operating income
amortisation of intangible assets and depreciation of property, plant and equipment   depreciation of property, plant and equipment   depreciation of machinery -> depreciation of property, plant and equipment
financial income and expenses   other interest income and related income   revenues from other receivabl

### Mapping data to ledger

In [51]:
from financial_ledger import financial_ledger as ledger
import jellyfish as jf
from jellyfish import jaro_winkler_similarity as jws
from tqdm import tqdm

In [52]:
# functions for calculating the Jaro-Winkler similarity for word matching

def add_matching_info_to_df(df, ledger):
    # Initialize lists to hold match results
    best_matches = []
    match_scores = []
    
    for index, row in df.iterrows():
        description = row['Description'].lower()
        max_score = 0
        best_match = ""
        for key, ledger_entry in ledger.items():
            # Iterate through both 'English' and 'Dutch' lists if they exist
            for lang in ['English', 'Dutch']:
                if lang in ledger_entry:  # Check if the language key exists
                    for term in ledger_entry[lang]:
                        score = jf.jaro_winkler(description, term.lower())
                        if score > max_score:
                            max_score = score
                            best_match = key
        # Append match result or indicate no match found
        if max_score > 0.7:
            best_matches.append(best_match)
            match_scores.append(max_score)
        else:
            best_matches.append("No match found")
            match_scores.append(max_score)
    
    # Add the match results to the DataFrame
    df['Best Match_JW'] = best_matches
    df['Match Score_JW'] = match_scores

add_matching_info_to_df(df_parsed, ledger)

# Now df_parsed contains two new columns: 'Best Match' and 'Match Score'
display(df_parsed)

#display only the rows with no match found
# df_no_match = df_parsed[df_parsed['Best Match_JW'] == "No match found"]
# display(df_no_match)


,Description,2022,2021,Category,Best Match_JW,Match Score_JW
0,"page 2 momo medical holding b.v., delft consol...",44292,13719,assets,No match found,0.675170
1,other tangible assets,784641,214361,other tangible assets,Non Current Assets,1.000000
2,"total of property, plant and equipment",828933,228080,"total of property, plant and equipment",Total of Non Current Assets,0.833298
3,total of non-current assets,828933,228080,total of non-current assets,Total of Non Current Assets,1.000000
4,current assets inventories raw materials a...,266885,63896,current assets inventories raw materials a...,Current Receivables,0.846762
...,...,...,...,...,...,...
98,,537558,537558,,No match found,0.000000
99,,690682,690682,,No match found,0.000000
100,2017/2018,321248,321248,2017/2018,No match found,0.000000
101,total,2236218,1524943,total,Total of Assets,0.876923


In [ ]:
# NOTE: do not run unless you want to display all rows
# this is used to display all rows in the dataframe when checking output, not necessary to run as it makes the output very long, only for testing/checking purposes

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto-detect the display width
pd.set_option('display.max_colwidth', -1)  # Display full width of columns

display(df_parsed)

In [54]:
#testing cosine similarity

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np


# Combine all texts to build the vocabulary for vectorization
all_texts = list(df_parsed['Category']) + [item for sublist in ledger.values() for lang in sublist if lang in ['English', 'Dutch'] for item in sublist[lang]]

vectorizer = TfidfVectorizer().fit(all_texts)
description_vectors = vectorizer.transform(df_parsed['Category'])

# Prepare and vectorize ledger entries
ledger_entries = [item for sublist in ledger.values() for lang in sublist if lang in ['English', 'Dutch'] for item in sublist[lang]]
ledger_vectors = vectorizer.transform(ledger_entries)

# Calculate cosine similarity between description vectors and ledger vectors
similarity_scores = cosine_similarity(description_vectors, ledger_vectors)

# Determine the best match for each description based on the highest cosine similarity score
best_matches = [ledger_entries[np.argmax(row)] if max(row) > 0.7 else "No match found" for row in similarity_scores]
df_parsed['Best Match_Cosine'] = best_matches
df_parsed['Highest Match Score_Cosine'] = [max(row) for row in similarity_scores]

#display(df_parsed)


,Description,2022,2021,Category,Best Match_JW,Match Score_JW,Best Match_Cosine,Highest Match Score_Cosine
0,"page 2 momo medical holding b.v., delft consolidated balance sheet as at 31 december 2022 after appropriation of results eur eur assets non-current assets property, plant and equipment machinery",44292,13719,assets,No match found,0.675170,No match found,0.625823
1,other tangible assets,784641,214361,other tangible assets,Non Current Assets,1.000000,other tangible assets,1.000000
2,"total of property, plant and equipment",828933,228080,"total of property, plant and equipment",Total of Non Current Assets,0.833298,"property, plant, and equipment",0.891444
3,total of non-current assets,828933,228080,total of non-current assets,Total of Non Current Assets,1.000000,total of non-current assets,1.000000
4,current assets inventories raw materials and consumables,266885,63896,current assets inventories raw materials and consumables,Current Receivables,0.846762,No match found,0.646660
5,inventories under construction,42450,25064,inventories under construction,Inventories under Construction,1.000000,inventories under construction,1.000000
6,finished and trade goods,125005,61284,finished and trade goods,Finished and Trade Goods,1.000000,finished and trade goods,1.000000
7,total of inventories,434340,150244,total of inventories,Total of Inventories,1.000000,total of inventories,1.000000
8,receivables trade receivables,182558,229329,receivables trade receivables,Current Receivables,0.966407,receivables trade receivables,1.000000
9,current receivables from other legal entities and companies with a participating interest in the legal entity or from participating interests of the legal entity,50,50,current receivables from other legal entities and companies with a participating interest in the legal entity or from participating interests of the legal entity,Current Receivables,0.910559,current receivables from other legal entities and companies with a participating interest,0.727810


## 3. Creating output

In [85]:
# if in df_parsed, best match JR and best match cosine  = "not found" or "not applicable", delete the row
# TODO: check if when there is no match from one method, if there is one from the other 

#if there is no match in 'Best Match_Cosine', replace it with the match from 'Best Match_JW'
df_parsed['Best Match_Cosine'] = np.where(df_parsed['Best Match_Cosine'] == "No match found", df_parsed['Best Match_JW'], df_parsed['Best Match_Cosine'])

df_parsed = df_parsed[df_parsed['Best Match_Cosine'] != "No match found"]
df_parsed = df_parsed[df_parsed['Best Match_Cosine'] != "not applicable"]

# if categories are found, make a new df with catgegory name, year 1 value,  year 2 value, output info from the ledger like id and other categories based on best match
df_output = df_parsed[['Category', company_information_df['Year 1'][0], company_information_df['Year 2'][0], 'Best Match_JW', 'Match Score_JW', 'Best Match_Cosine', 'Highest Match Score_Cosine']]

# based on the best match in the ledger, make new columns with balance, id, category, and statement type from the ledger for each row in df_output
def add_ledger_info_to_df(df, ledger):
    # Initialize lists to hold match results
    balance = []
    id = []
    category = []
    statement_type = []
    postencode= []
     
    for index, row in df.iterrows():
        best_match = row['Best Match_JW']
        for key, ledger_entry in ledger.items():
            if key == best_match:
                balance.append(ledger_entry['balance'])
                id.append(ledger_entry['id'])
                category.append(ledger_entry['category'])
                statement_type.append(ledger_entry['statement_type'])
                postencode.append(ledger_entry['postencode'])
    
    # Add the match results to the DataFrame
    df_output['Balance'] = balance
    df_output['ID'] = id
    df_output['Category'] = category
    df_output['Statement Type'] = statement_type
    df_output['Postencode'] = postencode

    
add_ledger_info_to_df(df_output, ledger)
#display(df_output)
df_output['Postencode'] = df_output['Postencode'].astype('int64')




In [86]:
# import excel file from path: /Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/20231212_kvk_finposten_dictionary - Antwoorden.xlsx
kvk_file_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/20231212_kvk_finposten_dictionary - Antwoorden.xlsx'
kvk_df = pd.read_excel(kvk_file_path, sheet_name='Postencodes')
# leave only the first 5 columns
kvk_df = kvk_df.iloc[:, :5]

#display(kvk_df)


In [87]:
#join output df with kvk_df on the postencode column
df_output = df_output.merge(kvk_df, how='left', left_on='Postencode', right_on='Postencode')

#display(df_output)

In [ ]:
#delete the Match Score_JW	Best Match_Cosine	Highest Match Score_Cosine, Category, Balance, and ID columns 
df_output = df_output.drop(columns=['Match Score_JW', 'Best Match_Cosine', 'Highest Match Score_Cosine', 'Category', 'Balance', 'ID', "Statement Type"])
# display NaN as '-'
df_output = df_output.fillna('-')

# display the dataframe
#display(df_output)


In [97]:
# Melting the DataFrame to have 'year' and 'value' columns
df_melted = df_output.melt(id_vars=[ "Postencode", 'Best Match_JW', "Hoofd posten code:", "Balans/res.rek. indicatie:", "Debet/credit indicatie: ", "Omschr. postencode: "],
                           var_name="Year", value_name="Value")


display(df_melted)

,Postencode,Best Match_JW,Hoofd posten code:,Balans/res.rek. indicatie:,Debet/credit indicatie:,Omschr. postencode:,Year,Value
0,120,Non Current Assets,100.0,B,D,materiële vaste activa,2022,784641
1,1,Total of Non Current Assets,-,-,-,-,2022,828933
2,1,Total of Non Current Assets,-,-,-,-,2022,828933
3,210,Inventories under Construction,200.0,B,D,voorraad,2022,42450
4,210,Finished and Trade Goods,200.0,B,D,voorraad,2022,125005
5,1,Total of Inventories,-,-,-,-,2022,434340
6,299,Current Receivables,200.0,B,D,overige vlottende activa,2022,182558
7,299,Current Receivables,200.0,B,D,overige vlottende activa,2022,50
8,299,Current Receivables,200.0,B,D,overige vlottende activa,2022,8647
9,299,Prepayments and Accrued Income,200.0,B,D,overige vlottende activa,2022,9046


## 5. Data Validation


In [ ]:
#Consistency check ...

# Balance sheet validation rule: Assets = Liabilities + Equity
if total_assets == total_liabilities + equity:
    print("The balance sheet balances.")
else:
    print("There is a discrepancy in the balance sheet.")
    